In [2]:
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.applications.mobilenet import preprocess_input

In [3]:
import os
import datetime as dt
import numpy as np
import cv2
from sklearn.svm import SVC

# package to save model
import pickle

In [4]:
TRAIN_DIR = "/content/drive/MyDrive/DATA/Outputt/face"
TEST_DIR = "/content/drive/MyDrive/DATA/Test_dataset/final"
MODEL_DIR = '/content/drive/MyDrive/DATA/Models'

In [6]:
pip install deepface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 805.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 10.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 9.3 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116934 sha256=3196043ec8427f758086a7b4bd705cce97e347f79561cc25d1376eb91d73dae6
  Stored in directory: /root/.cache/pip/wheels/90/d4/f7/9404e5db0116bd4d43e5666eaa3e70ab53723e1e3ea40c9a95
Successfully built fire


In [7]:
from deepface import DeepFace
deepface_model = DeepFace.build_model('Facenet512')

Directory  /root /.deepface created
Directory  /root /.deepface/weights created
facenet512_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/facenet512_weights.h5
To: /root/.deepface/weights/facenet512_weights.h5
100%|██████████| 95.0M/95.0M [00:01<00:00, 77.7MB/s]


In [12]:
from deepface.commons import functions

def extract_features(data, model):
    print("Extracting features ...")
    features = []
    for image_data in data:
        resized_image = cv2.resize(image_data, (160, 160))
        # Normalize the image
        normalized_image = resized_image / 255.0
        # Expand dimensions to add the batch size
        expanded_image = np.expand_dims(normalized_image, axis=0)
        try:
            feature = model.predict(expanded_image)
            features.append(feature.flatten())
        except Exception as e:
            print("Error processing image:", e)
    print("Completed.")
    return np.array(features)


# Load images from given DIR, where each subdirectory name is the label
def load_data_from_dir(DATA_DIR):
    print("Loading Data from dir:", DATA_DIR)
    subdirs = [d for d in os.listdir(DATA_DIR) if os.path.isdir(os.path.join(DATA_DIR, d))]
    images = []
    labels = []

    for subdir in subdirs:
        subdir_path = os.path.join(DATA_DIR, subdir)
        files = os.listdir(subdir_path)
        for file in files:
            filepath = os.path.join(subdir_path, file)
            print(filepath)
            image = cv2.imread(filepath)
            if image is not None:
                image = image.astype(np.float32)
                images.append(image)
                labels.append(subdir)  # The label is the subdirectory name
            else:
                print("Failed to load image:", file)

    X = np.array(images)
    y = np.array(labels)
    print("Loaded:", len(images), "images.")
    return X, y

import os
import cv2
import numpy as np

def load_test(DATA_DIR, file_path):
    images = []
    labels = []
    label_dict = {}

    # Load labels into a dictionary
    with open(file_path, 'r') as file:
        for line in file:
            label = line.strip().split(' ')
            label_dict[label[0]] = label[1]

    print("Loading Data from dir:", DATA_DIR)

    # List all files in DATA_DIR
    files_in_dir = os.listdir(DATA_DIR)

    for filename in files_in_dir:
        if filename.endswith('.jpeg') or filename.endswith('.jpg'):
            # Extracting the identifier from the filename (e.g., '12' from '12_0.jpeg')
            file_number = filename.split('_')[0]

            if file_number in label_dict:
                full_path = os.path.join(DATA_DIR, filename)
                image = cv2.imread(full_path)

                if image is not None:
                    print("Image loaded successfully:", filename)
                    image = cv2.resize(image, (224, 224))
                    image = image.astype(np.float32)
                    images.append(image)
                    labels.append(label_dict[file_number])
                else:
                    print(f"Failed to load image: {full_path}")
            else:
                print(f"No label found for image: {filename}")

    X = np.array(images)
    y = np.array(labels)
    print("Loaded:", len(images), "images with", len(labels), "labels.")
    return X, y



def get_now_string():
    return dt.datetime.now().__str__().replace(" ", "_").replace(":",'_').replace('-','_').replace('.', '_')


# Save trained model to file
def save_model_to_file(model, now_time, model_dir,  prefix=''):
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)

    model_ser_file_name = prefix + 'model_ser_' + os.path.basename(__file__).split('.')[0] + '_'+ now_time + '.pkl'
    save_path = os.path.join(model_dir, model_ser_file_name)
    print("Saving model in file:", save_path)
    pickle.dump(model, open(save_path, 'wb'))
    print("Completed.")

In [14]:
test_labels

array(['Pooja', 'Aparnaa', 'Haochen', 'Dheeraj', 'Ruthu', 'Zhiyu',
       'Varshit', 'Divyanth', 'Husain', 'Xinze', 'Varshit', 'Dheeraj',
       'Qihui', 'ShreyaDevendra', 'Zhiyu', 'Yash', 'Madhuri', 'Peiqi',
       'Pooja', 'Haochen', 'Ruthu', 'Yash', 'Shubham', 'Husain', 'Qihui',
       'Varshit', 'Zhiyu', 'Divyanth', 'Peiqi', 'Xinze', 'Haochen',
       'Dheeraj', 'Griffin', 'Yash', 'Varshit', 'Aparnaa', 'Zexin',
       'Pooja', 'Ruthu', 'Samyuktha', 'Madhuri', 'Shubham', 'Husain',
       'Haisong', 'Xinze', 'Aparnaa', 'ShreyaDevendra', 'Divyanth',
       'Yash', 'Ruthu', 'Pooja', 'Samyuktha', 'Haochen', 'Husain',
       'Zhiyu', 'Griffin', 'Peiqi', 'Zhiyu', 'Samyuktha',
       'ShreyaDevendra', 'Xinze', 'Varshit', 'Qihui', 'Aparnaa', 'Husain',
       'Peiqi', 'Yash', 'Haisong', 'Divyanth', 'Pooja', 'Shreyas',
       'Kaiyue', 'Prachi', 'Yuhang', 'Zexin', 'Chen', 'ShreyaChinthala',
       'Divyanth', 'Chen', 'Wei', 'Huiyu', 'Qihui', 'Yuhang', 'Huiyu',
       'Zexin', 'Samyuktha', 'An

In [13]:
# Load data
print("LOADING DATA ...")
train_data, train_labels = load_data_from_dir(TRAIN_DIR)
test_data, test_labels = load_test(TEST_DIR,'/content/drive/MyDrive/DATA/Test_dataset/final/labels.txt')
print("Completed")

LOADING DATA ...
Loading Data from dir: /content/drive/MyDrive/DATA/Outputt/face
/content/drive/MyDrive/DATA/Outputt/face/Kimsong/Capture7_face0.PNG
/content/drive/MyDrive/DATA/Outputt/face/Kimsong/Screenshot 2023-09-26 at 12.40.58 PM_face0.png
/content/drive/MyDrive/DATA/Outputt/face/Kimsong/Capture_face0.PNG
/content/drive/MyDrive/DATA/Outputt/face/Kimsong/Capture4_face0.PNG
/content/drive/MyDrive/DATA/Outputt/face/Kimsong/Capture2_face0.PNG
/content/drive/MyDrive/DATA/Outputt/face/Kimsong/Capture6_face0.PNG
/content/drive/MyDrive/DATA/Outputt/face/Kimsong/Capture8_face0.PNG
/content/drive/MyDrive/DATA/Outputt/face/Kimsong/Capture1_face0.PNG
/content/drive/MyDrive/DATA/Outputt/face/Anirudh/Screenshot 2023-10-21 at 9.59.56 PM_face0.png
/content/drive/MyDrive/DATA/Outputt/face/Anirudh/0921-ClassPhotos _face0.png
/content/drive/MyDrive/DATA/Outputt/face/Anirudh/0919-ClassPhotos_face0.png
/content/drive/MyDrive/DATA/Outputt/face/Anirudh/9-28_face0.png
/content/drive/MyDrive/DATA/Outputt/

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
# TRAIN
print("*****TRAINING*****")
train_features = extract_features(train_data, deepface_model)  # Use the DeepFace model here
print("training, please wait ...")
model = SVC()
model.fit(train_features, train_labels)

*****TRAINING*****
Extracting features ...
1/1 [==============================] - 0s 102ms/step
Completed.
training, please wait ...


SVC()

In [16]:
from datetime import datetime

def get_now_string():
    return datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

In [17]:
import joblib

def save_model_to_file(model, timestamp, directory):
    filename = f'model_{timestamp}.pkl'
    filepath = os.path.join(directory, filename)
    joblib.dump(model, filepath)
    print(f"Model saved to {filepath}")

# Assuming 'model' is your trained SVC model
svm_model = SVC()
svm_model.fit(train_features, train_labels)  # Ensure train_features are extracted using DeepFace

# Assuming 'MODEL_DIR' is the directory where you want to save the model
MODEL_DIR = '/content/drive/MyDrive/DATA/Models'

# Save the model
now_time_string = get_now_string()
save_model_to_file(svm_model, now_time_string, MODEL_DIR)


Model saved to /content/drive/MyDrive/DATA/Models/model_2023-12-04_00-35-14.pkl


In [18]:
# TEST
print("*****TESTING*****")
test_features = extract_features(test_data, deepface_model)  # Use DeepFace model here

# Make predictions and calculate accuracy
predictions = svm_model.predict(test_features)  # Use 'svm_model' to make predictions
accuracy = np.mean(predictions == test_labels.flatten())
print("*****RESULTS*****")
print(f"Accuracy: {accuracy * 100}%")

*****TESTING*****
Extracting features ...
1/1 [==============================] - 0s 111ms/step
Completed.
*****RESULTS*****
Accuracy: 94.53125%
